In [3]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [4]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.55.4
!uv pip install --no-deps trl==0.22.2

In [1]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Ahmed3310/llama-3-reasoning-16bit",
    load_in_4bit = True,         # ✅ 4-bit quantization
    fast_inference = False,      # ❌ vLLM not supported in 4-bit
    torch_dtype = "auto",        # auto usually picks float16
    device_map = "auto",
)

FastLanguageModel.for_inference(model)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

INFO 10-11 14:14:40 [__init__.py:244] Automatically detected platform cuda.
ERROR 10-11 14:14:42 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.1: Fast Llama patching. Transformers: 4.55.4. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm

In [5]:
import re # Import the re module

FastLanguageModel.for_inference(model)

# Define the system prompt (for structured analysis)
analysis_start = "<start_analysis>"
analysis_end = "</end_analysis>"
system_prompt = f"""You are an analyst given an article and associated entities.
Analyze the article and produce:
- Concise Summary: A brief overview.
- Expanded Analysis: A deeper analysis.
- Impact Forecast: List impacts with type, horizon (SHORT_TERM/MEDIUM_TERM/LONG_TERM), magnitude (LOW/MEDIUM/HIGH), probability (0-1), and rationale.
- Scenarios: List possible outcomes with probability (0-1) and description.
- Confidence: Overall confidence score (0-1).
- Evidence Refs: List of reference IDs.

Place your full analysis between {analysis_start} and {analysis_end}.
Output in plain text with sections like 'Concise Summary:', 'Impact Forecast:', etc."""

# Function to format entities
def format_entities(entities):
    ent_str = "Associated Entities:\n"
    for ent in entities:
        ent_str += f"- ID: {ent['entity_id']}, Name: {ent['name']}, Type: {ent['type']}, Profile: {ent['profile_text']}, Last Updated: {ent['last_updated']}\n"
    return ent_str

# Sample test input (Salesforce-Slack acquisition)
test_input = {
    "article_id": "A1021",
    "published_at": "2020-12-02T14:30:00Z",
    "article_text": """SAN FRANCISCO – In a landmark deal poised to reshape the future of work, enterprise software giant Salesforce announced Tuesday its definitive agreement to acquire workplace collaboration platform Slack Technologies for approximately $27.7 billion in cash and stock. The move marks Salesforce's largest-ever acquisition and positions it to compete directly with Microsoft in the booming market for team communication tools.

The acquisition brings together Salesforce's dominant Customer 360 platform with Slack, one of the most beloved communication tools in the tech industry. Salesforce CEO Marc Benioff hailed the combination as a 'match made in heaven,' stating that together, 'Salesforce and Slack will shape the future of enterprise software and transform the way everyone works in the all-digital, work-from-anywhere world.'

Under the terms of the deal, Slack shareholders will receive $26.79 in cash and 0.0776 shares of Salesforce common stock for each Slack share. This represents a significant premium on Slack's recent trading price on the New York Stock Exchange. Stewart Butterfield, Slack's CEO and co-founder, is expected to continue leading the Slack unit within Salesforce upon the deal's closure, which is anticipated in the second quarter of Salesforce’s fiscal year 2022.

'This is the most strategic combination in the history of software,' Butterfield said in a statement. 'I can’t wait to get going.'

The deal is a clear challenge to Microsoft, whose Teams product has seen explosive growth, often bundled with its popular Office 365 suite. By integrating Slack's channel-based messaging platform, Salesforce aims to create a comprehensive 'operating system for your workday,' deeply embedding communication within its core sales, service, and marketing products. Analysts believe this integration could create a stickier ecosystem for Salesforce customers, making it harder for competitors to poach clients. The transaction is subject to approval by Slack stockholders and regulatory authorities.""",
    "entities": [
        {"entity_id": "B-LOC-1", "last_updated": "2020-12-02", "name": "SAN FRANCISCO", "profile_text": "A major city in Northern California, United States, known as a global center of technology and innovation. It is the headquarters for numerous major tech companies, including Salesforce.", "type": "place"},
        {"entity_id": "B-ORG-2", "last_updated": "2020-12-02", "name": "Salesforce", "profile_text": "An American cloud-based software company headquartered in San Francisco, California. It provides customer relationship management (CRM) service and also sells a complementary suite of enterprise applications focused on customer service, marketing automation, analytics, and application development. Founded by Marc Benioff in 1999.", "type": "company"},
        {"entity_id": "B-ORG-3", "last_updated": "2020-12-02", "name": "Slack Technologies", "profile_text": "A software company that developed the Slack business communication platform. Founded by Stewart Butterfield, Eric Costello, Cal Henderson, and Serguei Mourachov in 2009. Headquartered in San Francisco, California. The company went public via a direct listing on the New York Stock Exchange in June 2019.", "type": "company"},
        {"entity_id": "B-ORG-4", "last_updated": "2020-12-02", "name": "Microsoft", "profile_text": "A multinational technology corporation headquartered in Redmond, Washington. It develops, manufactures, licenses, supports, and sells computer software, consumer electronics, personal computers, and related services. Its Microsoft Teams product is a direct competitor to Slack.", "type": "company"},
        {"entity_id": "B-PER-5", "last_updated": "2020-12-02", "name": "Marc Benioff", "profile_text": "An American internet entrepreneur, author, and philanthropist. He is the founder, chairman, and CEO of Salesforce, an enterprise cloud computing company. Known for his advocacy for 'compassionate capitalism.'", "type": "person"},
        {"entity_id": "B-ORG-6", "last_updated": "2020-12-02", "name": "New York Stock Exchange", "profile_text": "An American stock exchange in the Financial District of Lower Manhattan in New York City. It is by far the world's largest stock exchange by market capitalization of its listed companies. Slack (ticker: WORK) began trading on the NYSE in 2019.", "type": "organization"},
        {"entity_id": "B-PER-7", "last_updated": "2020-12-02", "name": "Stewart Butterfield", "profile_text": "A Canadian billionaire businessman and co-founder of the photo-sharing website Flickr and the team-messaging application Slack. He serves as the CEO of Slack Technologies.", "type": "person"}
    ]
}

# Format the test prompt
user_prompt = f"""Article ID: {test_input['article_id']}
Published: {test_input['published_at']}

{format_entities(test_input['entities'])}

Article Text: {test_input['article_text']}

Analyze this article."""

prompt = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Convert prompt to text for tokenization (Llama-style chat format)
prompt_text = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)

# Tokenize the input
inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)

# Generate output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=2048,  # Allow longer generation for full analysis
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

# Extract analysis between tags
match = re.search(rf"{re.escape(analysis_start)}\s*(.+?)\s*{re.escape(analysis_end)}", generated_text, re.DOTALL | re.IGNORECASE)
if match:
    analysis = match.group(1).strip()
    print("\nGenerated Analysis:")
    print(analysis)
else:
    print("\nRaw Generated Output (no tags found):")
    print(generated_text)


Raw Generated Output (no tags found):
<start_analysis>

**Concise Summary:**
Salesforce has agreed to acquire Slack Technologies for approximately $27.7 billion in cash and stock, creating a comprehensive platform for team communication and collaboration. The deal positions Salesforce to compete directly with Microsoft's Teams product, and analysts believe the integration could create a stickier ecosystem for Salesforce customers.

**Expanded Analysis:**

The acquisition of Slack by Salesforce is a strategic move to expand its customer relationship management (CRM) capabilities and compete in the growing market for team communication tools. Salesforce's dominant Customer 360 platform, which provides customer service, marketing automation, analytics, and application development, will be combined with Slack's channel-based messaging platform to create a comprehensive 'operating system for your workday.'

The deal is significant for several reasons:

1. **Market size:** The team communic